# **BCB 546x: Python Assignment**

##**Description**
###This notebook documents the script needed to complete the code provided in "sequence_translate.py" and documentation needed to understand the code. The tasks include: (1) translating cytochrome-b sequences to amino acids for each of the 12 species of penguins, (2) computing summaries of the amino acid molecular weight and GC content of these sequences, and (3) data visualization. 

##**Dependencies**
###The file(s) needed to run the script is/are: "sequence_translate.py", "penguins_mass.csv", and "penguins_cytb.fasta". 
###The package(s) need to run the script is/are: BioPython, pandas, dash, dash_bio, altair

##**Author**
###Jeniffer Louise Perea-Lopez

##**Date**
###Created: Tuesday, May 2, 2023 at 1:28 PM
###Modified: Friday, May 5, 2023 at 10:34 PM

In [3]:
#Install packages
%pip install biopython #to install Biopython
%pip install dash #to install Dash
%pip install dash-bio #to install dash_bio
%pip install altair vega_datasets #to install altair

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


In [ ]:
#Import python and Biopython packages and modules
from Bio import SeqIO #import Biopython package SeqIO
from Bio.Data import CodonTable #import Biopython package CodonTable
from Bio import Seq #import Biopython package Seq
from Bio.SeqUtils.ProtParam import ProteinAnalysis #import ProteinAnalysis from ProtParam module
from Bio.SeqUtils import GC #import GC from Bio.SeqUtils module
import pandas as pd #import pandas package
import matplotlib.pyplot as plt #import matplotlib.pyplot package
import seaborn as sns #import seaborn package
import altair as alt #import altair package

#-- Functions --#

##**Function `get_sequences_from_file(fasta_fn)`**
###**Description:** Pulls sequence data and returns species names and corresponding sequence data from FASTA file

###**Arguments:**
* `fata_fn`: where the .fasta file is placed 

###**Return:** Species name and sequence data for the different species found in the .fasta file

###**Example of usage:**
    >>>fasta_fn = "example_file.fasta"
    >>>get_sequences_from_file(fasta_fn)

###**Output:**
    {'Species name' : Seq('ATGCATGCATGCATGCATGCATGCATGCATGC...TAA')}

In [ ]:
def get_sequences_from_file(fasta_fn): #define new function mentioned above
    sequence_data_dict = {} #create an empty dictionary for sequence data
    for record in SeqIO.parse(fasta_fn, "fasta"): #for loop to identify sequences within .fasta file for each record 
        description = record.description.split() #assign function to split strings to variable
        species_name = description[1] + " " + description[2] #create species name from .fasta file
        sequence_data_dict[species_name] = record.seq #assign sequence to corresponding species name
    return(sequence_data_dict) #return dictionary

In [ ]:
#table of genetic code found in the mitochondria of all vertebrata (e.g., penguins)
mito_table = CodonTable.unambiguous_dna_by_name["Vertebrate Mitochondrial"]
print(mito_table)

##**Function `translate_sequences(string_nucleotides)`**
###**Description:** Translates a string of nucleotides to amino acids 

###**Arguments:**
* `string_nucleotides`: a string of nucleotides

###**Return:** A string of amino acids 

###**Example of usage:**
    >>>string_nucleotides = 'CCCGGTGTCGCTGCTCTCTCCGAGAAGAAGTAA'(NOTE: TAA is a stop codon)
    >>>translate_sequences(string_nucleotides)

###**Output:**
    'PGVAALSEKK' 

In [ ]:
def translate_sequences(string_nucleotides): #define new function mentioned above
    mito_table = CodonTable.unambiguous_dna_by_name["Vertebrate Mitochondrial"] #import codon table 
    aa_seq_string = "" #assign variable for amino acid sequences 
    for i in range (0, len(string_nucleotides), 3): #selects codons as multiples of 3 starting with 0
        codon = string_nucleotides[i] + string_nucleotides[i+1] + string_nucleotides[i+2] #define codon as a set of 3 nucleotides
        #NOTE: if the sequence has a stop codon at the end, it should be left off as it is a nonsense codon
        if codon == "TAA" or codon == "TAG" or codon == "AGA" or codon == "AGG": #identify stop codons 
            break # #terminate current loop at stop codon and resume execution for the next sequence
        else: #if loop did not encounter a break, statement will be executed after loop completes normally
            aa_seq_string += mito_table.forward_table[codon] #translate codons into amino acids 
    return(aa_seq_string) #retunrn amino acid sequence 

In [ ]:
#Run example of usage
string_nucleotides = 'CCCGGTGTCGCTGCTCTCTCCGAGAAGAAGTAA'
translate_sequences(string_nucleotides)

##**Function `alt_translate_sequences(string_nucleotides)`**
###**Description:** Alternative method to translates a string of nucleotides to amino acids by converting nucelotide sequence to a string of amino acids

###**Arguments:**
* `string_nucleotides`: a string of nucleotides

###**Return:** A string of amino acids 

###**Example of usage:**
    >>>string_nucleotides = 'CCCGGTGTCGCTGCTCTCTCCGAGAAGAAGTAA'NOTE: TAA is a stop codon
    >>>alt_translate_sequences(string_nucleotides)

###**Output:**
    'PGVAALSEKK' 

###**Reference:** https://biopython.org/wiki/Seq

In [ ]:
def alt_translate_sequences(string_nucleotides): #define new function mentioned above
    mito_table = CodonTable.unambiguous_dna_by_name["Vertebrate Mitochondrial"] #import codon table 
    aa_seq_string = str(Seq.translate(string_nucleotides, mito_table)) [:-1] #converts nucleotide sequence, 
    #except the last element associated with the stop codon, to string of amino acids
    return(aa_seq_string)

In [ ]:
#Run example of usage
string_nucleotides = 'CCCGGTGTCGCTGCTCTCTCCGAGAAGAAGTAA'
alt_translate_sequences(string_nucleotides)

##**Function `compute_molecular_weight(aa_seq)`**
###**Description:** Calculate the molecular weight of an amino acid sequence

###**Arguments:**
* `aa_seq`: a string of amino acids

###**Return:** molecular weight

###**Example of usage:**
    >>>aa_seq = 'PGVAALSEKK'
    >>>compute_molecular_weight(aa_seq)

###**Output:**
    999.1621
    
###**Reference:*** http://biopython.org/wiki/ProtParam

In [ ]:
def compute_molecular_weight(aa_seq): #define new function mentioned above
    analysed_seq = ProteinAnalysis(aa_seq) #convert string of amino acids to a sequence object using ProteinAnalysis
    return(analysed_seq.molecular_weight()) #compute the molecular weight from an amino acid sequence

In [ ]:
#Run example of usage 
aa_seq = 'PGVAALSEKK'
compute_molecular_weight(aa_seq)

##**Function `compute_GC_content(string_nucleotides)`**
###**Description:** Calculate the GC content (proportion of "G" and "C") of a DNA sequence

###**Arguments:**
* `string_nucleotides`: a string of nucleotides

###**Return:** percent GC content 

###**Example of usage:**
    >>>string_nucleotides = 'CCCGGTGTCGCTGCTCTCTCCGAGAAGAAGTAA'
    >>>compute_GC_content(string_nucleotides)

###**Output:**
    57.57575757575758
    
###**Reference:** https://biopython.org/docs/1.75/api/Bio.SeqUtils.html#Bio.SeqUtils.GC

In [ ]:
def compute_GC_content(string_nucleotides): #define new function mentioned above
    GC_content = GC(string_nucleotides) #calculate G+C content from string of nucleotides
    return(GC_content) #return GC content as %GC calculated against the full length

In [ ]:
#Run example of usage
string_nucleotides = 'CCCGGTGTCGCTGCTCTCTCCGAGAAGAAGTAA'
compute_GC_content(string_nucleotides)

#-- In the MAIN part of the script --#

In [ ]:
cytb_seqs = get_sequences_from_file("penguins_cytb.fasta") #pull sequence data from FASTA file and assign them to a dictionary 
penguins_df = pd.read_csv("penguins_mass.csv") #read .csv file into DataFrame, includes only data for body mass 
species_list = list(penguins_df.species) #creates a list of penguin species from DataFrame

In [ ]:
#Add two new columns to the penguin DataFrame: (1) molecular weight and (2) GC content 
#Set the value to 'NaN' to indicate that these cells are currently empty
penguins_df['molecular_weight'] = 'NaN'
penguins_df['GC_content'] = 'NaN'
print(penguins_df)

In [ ]:
#Write a for-loop that translates each sequence and also gets molecular weight and computes the GC content
#of each translated sequence and adds those data to DataFrame
row = 0 #define row as the first row '0'
for key, value in cytb_seqs.items(): 
    aa_seq = translate_sequences(value) #translate penguin DNA sequence to amino acid sequence
    molecular_weight = compute_molecular_weight(aa_seq) #compute MW of amino acid sequence
    penguins_df.loc[row, 'molecular_weight'] = molecular_weight #add MW to DataFrame
    GC_content = compute_GC_content(value) #compute %GC of DNA sequence 
    penguins_df.loc[row, 'GC_content'] = GC_content #add GC content to DataFrame
    row = row + 1 #define row for looping through rows in DataFrame
print(penguins_df)

In [ ]:
#Plot a bar-chart of the mass with the x-axes labeled with species names
penguins_df.plot(kind = 'bar', x = 'species', y = 'mass', ylabel = 'mass', color = '#0066cc', 
                 title = 'Mass of Penguin Species')

In [ ]:
#Using Vega visualization to highligh the smallest penguin species 
alt.Chart(penguins_df, title = "Mass of Penguin Species").mark_bar().encode(
    x='species:O',
    y="mass:Q",
    #highlight smallest penguin species
    color=alt.condition(
        alt.datum.species == 'Eudyptula minor',  #If the species is "Eudyptula minor" this test returns True,
        alt.value('#FF6600'),     #highlight a bar with orange (#FF6600)
        alt.value('#0066cc')   #And blue (#0066cc) for the rest of the bars
     )
).properties(width=600).configure_axis(
    labelFontSize=16,
    titleFontSize=16
).configure_title(fontSize=16)

###**Q1. What is the smallest penguin species?**
###*Eudyptula minor*

###**Q2. What is the geographical range of this species?**
###Found in the coastal waters of southern mainland Australia and Tasmania (https://australian.museum/learn/animals/birds/little-penguin-eudyptula-minor/)

In [ ]:
#Plot a visualization of the molecular weight (y-axis) as a function of GC-content (x-axis)
sns.scatterplot(x = "GC_content", y = "molecular_weight", data = penguins_df, s = 100, hue = "species")
plt.xlabel("GC content")
plt.ylabel("Molecular Weight")
plt.title("GC-content vs. Molecular Weight in Penguin Species")
plt.legend(bbox_to_anchor=(1.01, 1),
           borderaxespad=0)

In [ ]:
#Save the new DataFrame to a file called "penguins_mass_cytb.csv"
penguins_df.to_csv("penguins_mass_cytb.csv")

###**BONUS:What else can we do with this dataset in python?**
####Objective 1: Align multiple sequences within a FASTA file
####Reference: https://biopython.org/docs/1.76/api/Bio.Align.Applications.html

####Objective 2: Visualize sequence alignments using dash_bio
####Reference: https://dash.plotly.com/dash-bio/alignmentchart

In [ ]:
from Bio.Align.Applications import MuscleCommandline

In [ ]:
def align_seqs(fasta_fn): #define new function for sequence alignment
    seqs = SeqIO.parse(fasta_fn, 'fasta') #identify sequences in FASTA file 
    SeqIO.write(seqs, "align_seqs.fasta", "fasta") #write FASTA file with aligned sequences
    muscle_cmdline = MuscleCommandline(input = seqs, diags = True, maxiters = 1, 
                                       out = "align_seqs.fasta")
    #use multiple alignment program MUSCLE to align sequences and output FASTA file
    return('Alignment Complete!') #output message

In [ ]:
fasta_fn = "penguins_cytb.fasta"
align_seqs(fasta_fn)

In [ ]:
import urllib.request as urlreq
from dash import Dash, html
import dash_bio as dashbio

app = Dash(__name__)

#import aligned FASTA file from github
data = urlreq.urlopen('https://raw.githubusercontent.com/Jperea02/BCB546-PythonHW_Spring2023/main/align_seqs.fasta').read().decode('utf-8')

#run alignment chart and alignment viewer in html
app.layout = html.Div([
    dashbio.AlignmentChart(
        id='alignment-viewer',
        data=data
    ),
     html.Div(id='default-alignment-viewer-output')])

#run dash, defaul port is '8050'
if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)
#When code is running, it will provide a link to alignment viewer("Dash is running on http://127.0.0.1:8050/")
#I have not figured out a way to stop Dash from running other than completely exiting from Jupyter in terminal